In [1]:
import sqlalchemy as db
import pandas as pd

## Connecting to database

In [2]:
engine = db.create_engine('postgresql://localhost:5432/sql_challenge')
connection = engine.connect()
metadata = db.MetaData()

In [3]:
#to find all the tables for the database(tables are created using pgadmin)
engine.table_names()

/var/folders/ld/_spx9cwj74z4zm921xp1zssh0000gn/T/ipykernel_67316/3992685500.py:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['titles', 'departments', 'employees', 'dept_emp', 'dept_manager', 'salaries']

In [4]:
#getting the tables from database
titles = db.Table('titles', metadata, autoload=True, autoload_with=engine)
departments = db.Table('departments', metadata, autoload=True, autoload_with=engine)
employees = db.Table('employees', metadata, autoload=True, autoload_with=engine)
dept_emp = db.Table('dept_emp', metadata, autoload=True, autoload_with=engine)
dept_manager = db.Table('dept_manager', metadata, autoload=True, autoload_with=engine)
salaries = db.Table('salaries', metadata, autoload=True, autoload_with=engine)

In [5]:
# Print full table metadata
print(repr(metadata.tables['employees']))

Table('employees', MetaData(), Column('emp_no', INTEGER(), table=<employees>, primary_key=True, nullable=False), Column('emp_title_id', CHAR(length=10), ForeignKey('titles.title_id'), table=<employees>), Column('birth_date', DATE(), table=<employees>), Column('first_name', VARCHAR(length=20), table=<employees>), Column('last_name', VARCHAR(length=20), table=<employees>), Column('sex', CHAR(length=1), table=<employees>), Column('hire_date', DATE(), table=<employees>), schema=None)


In [6]:
# Print the column names
employees.columns.keys()

['emp_no',
 'emp_title_id',
 'birth_date',
 'first_name',
 'last_name',
 'sex',
 'hire_date']

In [7]:
#can also use c instead of columns
employees.c.keys()

['emp_no',
 'emp_title_id',
 'birth_date',
 'first_name',
 'last_name',
 'sex',
 'hire_date']

# Querying


### 1.

In [12]:
### sql
sql_1= '''SELECT e.emp_no, e.last_name, e.first_name, e.sex, s.salary 
       FROM employees AS e
       JOIN salaries AS s
       ON e.emp_no=s.emp_no'''

In [13]:
#using text
results_1=connection.execute(db.text(sql_1)).fetchall()
df_1 = pd.DataFrame(results_1, columns=(['Emp_No', 'Last_Name', 
                                         "First_Name",'Sex','Salary']))
df_1


Emp_No    Last_Name First_Name Sex  Salary
0        10005     Maliniak    Kyoichi   M   78228
1        10010     Piveteau  Duangkaew   F   72488
2        10011        Sluis       Mary   F   42365
3        10013       Terkki  Eberhardt   M   40000
4        10017    Bouloucos  Cristinel   F   71380
...        ...          ...        ...  ..     ...
300019  499948    Paludetto   Cordelia   M   40000
300020  499972       Leuchs     Katsuo   F   41002
300021  499985  Lukaszewicz       Gila   M   40000
300022  499987       Dusink      Rimli   F   52282
300023  499998      Breugel   Patricia   M   40000

[300024 rows x 5 columns]

### 2.

In [16]:
sql_2='''SELECT first_name, last_name, hire_date
        FROM employees
        WHERE hire_date BETWEEN '1986-01-01' AND '1986-12-31' '''


In [17]:
#using text
results_2=connection.execute(db.text(sql_2)).fetchall()
df_2 = pd.DataFrame(results_2, columns=(['first_name', 'last_name', 'hire_date']))
df_2

first_name    last_name   hire_date
0           Eran     Cusworth  1986-11-14
1          Bojan     Zallocco  1986-10-14
2          Nevio       Demizu  1986-05-18
3           Ziva       Vecchi  1986-07-03
4          Mohit        Speek  1986-01-14
...          ...          ...         ...
36145      Uriel     Heijenga  1986-06-30
36146      Ziyad  Constantine  1986-02-28
36147     Yishay   Maksimenko  1986-01-27
36148     Yannik        Ranai  1986-04-06
36149     Chenyi     Orlowska  1986-12-25

[36150 rows x 3 columns]

### 3.

In [18]:
sql_3='''SELECT  d.dept_no, d.dept_name, e.emp_no, e.last_name, e.first_name 
        FROM departments AS d
        JOIN dept_manager AS dm
        ON d.dept_no=dm.dept_no
        JOIN employees AS e
        ON dm.emp_no=e.emp_no'''

In [23]:
#using text
results_3=connection.execute(db.text(sql_3)).fetchall()
df_3 = pd.DataFrame(results_3, columns=(['dept_no', 'dept_name', 
                                         'emp_no', 'last_name', 'first_name']))
df_3

dept_no           dept_name  emp_no     last_name   first_name
0   d001                 Marketing  110022    Markovitch    Margareta
1   d001                 Marketing  110039      Minakawa     Vishwani
2   d002                   Finance  110085         Alpin         Ebru
3   d002                   Finance  110114    Legleitner        Isamu
4   d003           Human Resources  110183  Ossenbruggen      Shirish
5   d003           Human Resources  110228       Sigstam      Karsten
6   d004                Production  110303       Wegerle    Krassimir
7   d004                Production  110344         Cools       Rosine
8   d004                Production  110386        Kieras         Shem
9   d004                Production  110420      Ghazalie        Oscar
10  d005               Development  110511      Hagimont     DeForest
11  d005               Development  110567      DasSarma         Leon
12  d006        Quality Management  110725       Onuegbe    Peternela
13  d006        Quality Management  110765       Hofmeyr       Rutger
14  d006        Quality Management  110800       Quadeer       Sanjoy
15  d006        Quality Management  110854         Pesch         Dung
16  d007                     Sales  111035     Kaelbling  Przemyslawa
17  d007                     Sales  111133         Zhang        Hauke
18  d008                  Research  111400       Staelin         Arie
19  d008                  Research  111534        Kambil       Hilary
20  d009          Customer Service  111692   Butterworth        Tonny
21  d009          Customer Service  111784    Giarratana        Marjo
22  d009          Customer Service  111877      Spinelli      Xiaobin
23  d009          Customer Service  111939       Weedman      Yuchang

### 4.

In [21]:
sql_4='''SELECT e.emp_no, e.last_name, e.first_name, d.dept_name
        FROM employees AS e
        JOIN dept_emp AS de
        ON e.emp_no=de.emp_no
        JOIN departments AS d
        on de.dept_no=d.dept_no'''

In [22]:
#using text
results_4=connection.execute(db.text(sql_4)).fetchall()
df_4 = pd.DataFrame(results_4, columns=(['emp_no', 'last_name', 
                                         'first_name', 'dept_name']))
df_4

emp_no    last_name first_name           dept_name
0        10005     Maliniak    Kyoichi     Human Resources
1        10010     Piveteau  Duangkaew          Production
2        10010     Piveteau  Duangkaew  Quality Management
3        10011        Sluis       Mary    Customer Service
4        10013       Terkki  Eberhardt     Human Resources
...        ...          ...        ...                 ...
331598  499948    Paludetto   Cordelia          Production
331599  499972       Leuchs     Katsuo         Development
331600  499985  Lukaszewicz       Gila            Research
331601  499987       Dusink      Rimli               Sales
331602  499998      Breugel   Patricia             Finance

[331603 rows x 4 columns]

### 5.

In [24]:
sql_5='''SELECT first_name, last_name, sex
        FROM employees
        WHERE first_name='Hercules' AND last_name LIKE'B%' '''

In [25]:
#using text
results_5=connection.execute(db.text(sql_5)).fetchall()
df_5 = pd.DataFrame(results_5, columns=(['first_name', 'last_name', 'sex']))
df_5

first_name      last_name sex
0    Hercules           Baer   M
1    Hercules          Biron   F
2    Hercules          Birge   F
3    Hercules      Bernatsky   M
4    Hercules        Berstel   F
5    Hercules           Bail   F
6    Hercules       Benantar   F
7    Hercules         Bodoff   M
8    Hercules        Bisiani   F
9    Hercules        Basagni   M
10   Hercules  Bernardinello   F
11   Hercules     Baranowski   M
12   Hercules           Bahr   M
13   Hercules     Benzmuller   M
14   Hercules          Biran   F
15   Hercules           Bain   F
16   Hercules        Buchter   M
17   Hercules        Brendel   F
18   Hercules       Barreiro   M
19   Hercules           Baak   M

### 6.

In [26]:
sql_6='''SELECT e.emp_no, e.last_name, e.first_name, e.sex, d.dept_name
        FROM employees AS e
        JOIN dept_emp AS de
        ON e.emp_no=de.emp_no
        JOIN departments AS d
        ON de.dept_no=d.dept_no
        WHERE d.dept_name='Sales' '''

In [27]:
#using text
results_6=connection.execute(db.text(sql_6)).fetchall()
df_6 = pd.DataFrame(results_6, columns=(['emp_no', 'last_name', 
                                         'first_name','sex','dept_name']))
df_6

emp_no    last_name first_name sex dept_name
0       10002       Simmel    Bezalel   F     Sales
1       10016  Cappelletti   Kazuhito   M     Sales
2       10034         Swan      Bader   M     Sales
3       10041       Lenart        Uri   F     Sales
4       10050       Dredge    Yinghua   M     Sales
...       ...          ...        ...  ..       ...
52240  499750       Melter       Maya   M     Sales
52241  499986        Ranta     Nathan   F     Sales
52242  499987       Dusink      Rimli   F     Sales
52243  499752      Stranks     Palash   M     Sales
52244  499988      Kleiser   Bangqing   F     Sales

[52245 rows x 5 columns]

### 7.

In [28]:
sql_7='''SELECT e.emp_no, e.last_name, e.first_name, e.sex, d.dept_name
        FROM employees AS e
        JOIN dept_emp AS de
        ON e.emp_no=de.emp_no
        JOIN departments AS d
        ON de.dept_no=d.dept_no
        WHERE d.dept_name='Sales' OR d.dept_name='Development' '''

In [29]:
#using text
results_7=connection.execute(db.text(sql_7)).fetchall()
df_7 = pd.DataFrame(results_7, columns=(['emp_no', 'last_name', 
                                         'first_name','sex','dept_name']))
df_7

emp_no  last_name first_name sex    dept_name
0        10001    Facello     Georgi   M  Development
1        10002     Simmel    Bezalel   F        Sales
2        10006    Preusig     Anneke   F  Development
3        10008   Kalloufi     Saniya   M  Development
4        10012  Bridgland   Patricio   M  Development
...        ...        ...        ...  ..          ...
137947  499911   Tagansky     Satosi   F  Development
137948  499914    Keirsey       Khoa   F  Development
137949  499919   Angiulli     Masako   M        Sales
137950  499920    Murtagh     Christ   M        Sales
137951  499924       Swan      Angus   M  Development

[137952 rows x 5 columns]

### 8.

In [30]:
sql_8='''SELECT last_name, COUNT(last_name) AS count
        FROM employees
        GROUP BY last_name
        ORDER BY count DESC'''

In [31]:
#using text
results_8=connection.execute(db.text(sql_8)).fetchall()
df_8 = pd.DataFrame(results_8, columns=(['last_name','count']))
df_8

last_name  count
0           Baba    226
1         Gelosh    223
2          Coorg    223
3         Farris    222
4        Sudbeck    222
...          ...    ...
1633        Zykh    148
1634  Guardalben    148
1635       Merro    147
1636    Sadowsky    145
1637    Foolsday      1

[1638 rows x 2 columns]

# Creating Table

In [ ]:
#creating new table employees
employees = db.Table('employees', metadata,
              db.Column('emp_no', db.Integer(), primary_key=True, nullable=False), 
              db.Column('emp_title_id', db.String(10), ForeignKey('titles.title_id')), 
              db.Column('birth_date', db.Date()),
              db.Column('first_name', db.String(255)),
              db.Column('last_name', db.String(255)),
              db.Column('sex', db.String(10)), 
              db.Column('hire_date', db.Date())  
              )


metadata.create_all(engine) #Creates the table